<a href="https://colab.research.google.com/github/Arthur-galvao/trabalho-tf-idf-algebra/blob/main/gpu_datasheet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### carregamento de dados

In [ ]:
import pandas as pd
import sys
import re

NOME_ARQUIVO_EXCEL = 'Graphics Card Database.xlsx'

PAGINAS_DESEJADAS = [
    'ATIAMD',
    'NVIDIA',
]

COLUNAS_DESEJADAS = [
    'GPU_NAME',
    'MEMORY SIZE (MB)',
    'Memory Type*',
    'Price at Launch (MSRP)',
    'Year of Release'
]

lista_dataframes_paginas = []

print(f"Iniciando a leitura do arquivo: '{NOME_ARQUIVO_EXCEL}'...")

try:
    with pd.ExcelFile(NOME_ARQUIVO_EXCEL) as xls:
        planilhas_no_arquivo = xls.sheet_names

    paginas_para_ler = [p for p in PAGINAS_DESEJADAS if p in planilhas_no_arquivo]
    paginas_faltando = [p for p in PAGINAS_DESEJADAS if p not in planilhas_no_arquivo]

    if paginas_faltando:
        print(f"\nAviso: As seguintes páginas não foram encontradas e serão ignoradas: {paginas_faltando}")

    if not paginas_para_ler:
        print(f"Erro: Nenhuma das páginas desejadas ({PAGINAS_DESEJADAS}) foi encontrada no arquivo.")
        sys.exit()

    print(f"\nProcessando as seguintes páginas: {paginas_para_ler}...")

    for nome_pagina in paginas_para_ler:
        print(f"  - Lendo '{nome_pagina}'...")

        df_pagina = pd.read_excel(NOME_ARQUIVO_EXCEL, sheet_name=nome_pagina)

        colunas_existentes_na_pagina = [col for col in COLUNAS_DESEJADAS if col in df_pagina.columns]

        if not colunas_existentes_na_pagina:
            print(f"    Aviso: Nenhuma das colunas desejadas foi encontrada na página '{nome_pagina}'. Página ignorada.")
            continue

        df_transformado = df_pagina[colunas_existentes_na_pagina]

        lista_dataframes_paginas.append(df_transformado)

    if not lista_dataframes_paginas:
        print("\nErro: Nenhuma página foi processada com sucesso. O resultado está vazio.")
        df_gpus = pd.DataFrame()
    else:
        print("\nJuntando todos os dados das páginas em uma única tabela...")

        df_gpus = pd.concat(lista_dataframes_paginas, ignore_index=True)

        print("\n--- Resultado Combinado (Primeiras 10 linhas) ---")
        display(df_gpus.head(10))

        print("\n--- Informações do DataFrame Inicial ---")
        df_gpus.info()

        ARQUIVO_SAIDA_CSV = 'gpu_dataset_combinado.csv'
        df_gpus.to_csv(ARQUIVO_SAIDA_CSV, index=False)
        print(f"\nDados combinados salvos com sucesso em '{ARQUIVO_SAIDA_CSV}'")


except FileNotFoundError:
    print(f"ERRO CRÍTICO: O arquivo '{NOME_ARQUIVO_EXCEL}' não foi encontrado.")
    print("Verifique se o nome do arquivo está correto e se ele está na mesma pasta do script.")
    df_gpus = pd.DataFrame()
except Exception as e:
    print(f"Ocorreu um erro inesperado durante o processo: {e}")
    df_gpus = pd.DataFrame()

### Engenharia de Features

In [ ]:
price_translation_map = {
    'Enthusiast': 'Entusiasta (Alto Desempenho)',
    'Highend': 'Alto Nível (Premium)',
    'Mainstream': 'Intermediário (Popular)',
    'Lower Mainstream': 'Intermediário Inferior',
    'Upper Mainstream': 'Intermediário Superior',
    'Budget': 'Entrada',
    'Extreme': 'Extremo',
}

df_gpus['Price at Launch (MSRP)'] = df_gpus['Price at Launch (MSRP)'].map(price_translation_map).fillna(df_gpus['Price at Launch (MSRP)'])

def categorize_year(year):
    if pd.isna(year):
        return "unknown"
    try:
        year = int(year)
        if year >= 2020:
            return "recente"
        elif year >= 2010:
            return "antiga"
        elif year < 2010:
            return "muito antigas"
    except ValueError:
        return "unknown"

df_gpus['year_category'] = df_gpus['Year of Release'].apply(categorize_year)
df_gpus['Year of Release'] = df_gpus['Year of Release'].astype(str)

def extract_gpu_components(gpu_name):
    if pd.isna(gpu_name):
        return ""

    gpu_name = str(gpu_name).upper()
    components = []

    if 'GTX' in gpu_name:
        components.append('GTX')
        match = re.search(r'GTX\s?(\d+)', gpu_name)
        if match:
            num_str = match.group(1)
            if len(num_str) > 0:
                if len(num_str) >= 2:
                    components.append(f"generation {num_str[:2]}")
                    components.append(f"model_id {num_str[2:]}")
                elif len(num_str) == 1:
                     components.append(f"generation {num_str}")


    elif 'RTX' in gpu_name:
        components.append('RTX')
        match = re.search(r'RTX\s?(\d+)', gpu_name)
        if match:
            num_str = match.group(1)
            if len(num_str) > 0:
                if len(num_str) >= 2:
                    components.append(f"generation {num_str[:2]}")
                    components.append(f"model_id {num_str[2:]}")
                elif len(num_str) == 1:
                     components.append(f"generation {num_str}")


    elif 'RADEON RX' in gpu_name:
        components.append('RADEON RX')
        match = re.search(r'RADEON\s?RX\s?(\d+)', gpu_name)
        if match:
            num_str = match.group(1)
            if len(num_str) > 0:
                if len(num_str) >= 2:
                    components.append(f"generation {num_str[:1]}")
                    components.append(f"model_id {num_str[1:]}")
                elif len(num_str) == 1:
                     components.append(f"generation {num_str}")

        if 'XT' in gpu_name:
            components.append('XT variant')
        if 'TI' in gpu_name:
            components.append('TI variant')


    return " ".join(components)

df_gpus['gpu_components'] = df_gpus['GPU_NAME'].apply(extract_gpu_components)

combined_text_list = []

for index, row in df_gpus.iterrows():
    gpu_name = str(row['GPU_NAME']) if pd.notna(row['GPU_NAME']) else "unknown"
    memory_type = str(row['Memory Type*']) if pd.notna(row['Memory Type*']) else "unknown"
    price_at_launch = str(row['Price at Launch (MSRP)']) if pd.notna(row['Price at Launch (MSRP)']) else "unknown"
    year_of_release = str(row['Year of Release']) if pd.notna(row['Year of Release']) else "unknown"
    year_category = str(row['year_category']) if pd.notna(row['year_category']) else "unknown"
    gpu_components = str(row['gpu_components']) if pd.notna(row['gpu_components']) else ""


    combined_text = f"{gpu_name} {memory_type} {price_at_launch} {year_of_release} {year_category} {gpu_components}"
    combined_text_list.append(combined_text)

df_gpus['combined_text'] = combined_text_list

print("\n--- DataFrame após Engenharia de Features (Primeiras 10 linhas) ---")
display(df_gpus.head(10))

print("\n--- Informações do DataFrame Final ---")
df_gpus.info()

### Processamento de Texto (TF-IDF) e Sistema de Recomendação

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df_gpus['combined_text'].fillna(''))

print("Matriz TF-IDF criada com sucesso!")
print(f"Shape da matriz TF-IDF: {tfidf_matrix.shape}")

def recommend_gpus(query, tfidf_matrix, df, tfidf_vectorizer, top_n=5):
    query_vector = tfidf_vectorizer.transform([query])
    cosine_sim = cosine_similarity(query_vector, tfidf_matrix)
    similarity_scores = list(enumerate(cosine_sim[0]))
    sorted_gpus = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

    print(f"\nRecomendações para a busca: '{query}'")
    print("-" * 30)

    count = 0
    for i, score in sorted_gpus:
        if count < top_n:
            gpu_name = df.iloc[i]['GPU_NAME']
            print(f"GPU: {gpu_name} | Score de Similaridade: {score:.4f}")
            count += 1
        else:
            break

    if count == 0:
        print("Nenhuma recomendação encontrada.")

while True:
    user_query = input("Digite sua busca por GPU (ou 'sair' para encerrar): ")
    if user_query.lower() == 'sair':
        break
    recommend_gpus(user_query, tfidf_matrix, df_gpus, tfidf_vectorizer)

### Como Rodar os Códigos
Para executar este notebook e obter as recomendações de GPU, siga estes passos:

1. **Certifique-se de que o arquivo 'Graphics Card Database.xlsx' esteja presente** no mesmo diretório deste notebook. O primeiro código o lerá e criará o arquivo `gpu_dataset_combinado.csv`.
2. **Execute as células de código em sequência.** Há três células de código principais: "Carregamento de Dados e Pré-processamento Inicial", "Engenharia de Features" e "Processamento de Texto (TF-IDF) e Sistema de Recomendação".
3. **A última célula iniciará o sistema de recomendação interativo.** Você será solicitado a digitar sua busca por GPU.
4. **Para encerrar o sistema**, digite 'sair'.